### 药企圈OpenAPI接口测试

#### 一、环境选择

In [ ]:
# coding=utf-8
import pandas as pd
from colorama import Fore,Back,Style
from api import servers,interfaces
from tester import testers

DISPLAY_CHOICE = True

if DISPLAY_CHOICE:
    df = pd.DataFrame(servers,index=[0])
    df = df.T
    df.columns=['url']
    print(Fore.GREEN+'----------------------------------------\n',df)
    df = pd.DataFrame(testers)
    df = df.T
    print(Fore.GREEN+'----------------------------------------\n',df)

In [ ]:
# --修改下面的名称来选择环境-----------------
envcode = 'test'
# --修改下面的名称来选择账号-----------------
testercode = 'tester01'
# ---------------------------------------

In [ ]:
server = servers[envcode]
print(Fore.BLUE+'----------------------------------------\n当前选择的环境是: %s --> %s' % (envcode,server))
tester = testers[testercode]
if tester['envcode'] !=envcode: 
    print(Fore.BLUE+'----------------------------------------\n当前选择的账号的环境不匹配: %s --> envcode:%s' % (testercode,tester['envcode']))
else:
    from colorama import Fore
    print(Fore.BLUE+'----------------------------------------\n当前选择的账号是: %s' % testercode)

#### 二、接口选择

In [ ]:
if DISPLAY_CHOICE:
    print(Fore.GREEN+'----------------------------------------\n',pd.DataFrame([{'可选择的接口类别':x} for x in interfaces.keys()]))

In [ ]:
# --修改下面的内容来选择接口类别--------------
interface_catagory = interfaces['crm_read']
# ---------------------------------------

In [ ]:
interface_content = interface_catagory['interface']
if DISPLAY_CHOICE:
    print(Fore.GREEN+'----------------------------------------\n',pd.DataFrame([{'可选择的接口':x} for x in interface_catagory['interface']]))

In [ ]:
# --修改下面的内容来选择接口-----------------
interface = interface_content['queryHospitalData']
# ---------------------------------------

In [ ]:
api_key = interface_catagory['api_key'][envcode]
api_secret = interface_catagory['api_secret'][envcode]

print(Fore.BLUE+'当前选择的接口是: [%s]\nurl = %s\nmethod = %s\nbody = %s \noutput = %s \nkey = %s\nsecret = %s' 
      % (interface['desc'],interface['url'],interface['method'],
         interface.get('body',False),interface.get('output',[]),
         api_key,api_secret))
pd.DataFrame(interface['params'],columns=['参数']).head(100)

#### 三、输入数据准备

In [ ]:
# 以下统一的参数无须配置:
# 1.appkey  2.nonce  3.time  4.sign
#
# --修改下面的内容来修改其他输入参数---------
params = {
    'companyId':tester['companyId'],
    'timeStamp':0,
    'pageIndex':1,
    'pageSize':100
}
# --修改下面的内容来修HTTP BODY参数--------
data = {
    
}

In [ ]:

def generate_nonce(len = 16, lowercase = True, uppercase = False, digits = True) :
    import random
    import string
    random.seed()
    chars = ''
    if lowercase: chars += string.ascii_lowercase
    if uppercase: chars += string.ascii_uppercase
    if digits: chars += string.digits
    return ''.join([random.choice(chars) for _ in range(len)])

def generate_time() :
    import time
    return int(time.time()*1000)

def generate_sign(params,secret):
    params_list = [key + '=' + str(value) + '&' for key, value in params.items() if value is not None or str(value) != '' ]
    params_list.sort()
    params_str = ''.join(params_list)
    params_str = params_str + 'key=' + secret
    params_str = md5(params_str.encode('utf-8'))
    return params_str
    
def md5(str):
    import hashlib
    m = hashlib.md5()   
    m.update(str)
    return m.hexdigest()

params['appKey'] = api_key
params['nonce'] = generate_nonce()
params['time'] = generate_time()
params['sign'] = generate_sign(params,api_secret)

df=pd.DataFrame(params,index=[0]).T
df.columns=['参数值']
df.head(100)

#### 四、执行接口

In [ ]:
import requests
if interface['method'] == 'GET':
    r = requests.get('http://' + server + interface['url'],params=params)
    print(Fore.BLUE+'[%s请求执行完成] --> %s' % (interface['method'],r.url))
elif interface['method'] == 'POST':
    if interface.get('body',False):
        r = requests.post('http://' + server + interface['url'],params=params,data=data)
    else:
        r = requests.post('http://' + server + interface['url'],params=params)
    print(Fore.BLUE+'[%s请求执行完成] --> %s' % (interface['method'],r.url))
else:
    print(Fore.BLUE+'[%s接口方法填写错误，请求未执行]' % interface['method'])

#### 五、输出数据查询

In [ ]:
import json
output = json.loads(r.text)
resultCode = output['resultCode']
if resultCode == 1:
    print(Fore.BLUE+'[请求成功]')
    for field in interface['output']:
        output = output[field]
    df=pd.DataFrame(output)
else:
    print(Fore.BLUE+output['请求失败：%s' % output['resultMsg']])
    pass
print(Fore.BLUE+'最前5条数据：')
df.head(5)

In [ ]:
if resultCode == 1:
    pass
print(Fore.BLUE+'最后5条数据：')
df.tail(5)